CARGA DE TABLAS A LA BASE DE DATOS

In [2]:
import pandas as pd
import mysql.connector

In [ ]:
# Cargar CSV´s

df_balance = pd.read_csv("../data/processed/balance/balance-electrico-limpio.csv")

df_demanda_evolucion = pd.read_csv("../data/processed/demanda/demanda-limpio.csv")
df_demanda_ire = pd.read_csv("../data/processed/demanda/ire-limpio.csv")

df_estructura_generacion = pd.read_csv("../data/processed/generacion/estructura-generacion-limpio.csv")

df_fronteras = pd.read_csv("../data/processed/intercambio/fronteras-limpio.csv")

In [132]:
df_balance.head()

,fecha,valor,tipo,energia,region
0,2019-01-01,53.722120,Hidráulica,Renovable,peninsular
1,2019-01-02,61.294768,Hidráulica,Renovable,peninsular
2,2019-01-03,82.981951,Hidráulica,Renovable,peninsular
3,2019-01-04,94.301940,Hidráulica,Renovable,peninsular
4,2019-01-05,58.822692,Hidráulica,Renovable,peninsular


In [133]:
df_demanda_evolucion.head()

,fecha,valor,indicador,region,año
0,2019-01-01,582.949806,Demanda,peninsular,2019
1,2019-01-02,742.199407,Demanda,peninsular,2019
2,2019-01-03,787.764963,Demanda,peninsular,2019
3,2019-01-04,796.010035,Demanda,peninsular,2019
4,2019-01-05,721.029651,Demanda,peninsular,2019


In [134]:
df_demanda_ire.head()

,fecha,valor,porcentaje,indicador,region,año
0,2019-01-01,123.969,0.541252,Índice general,peninsular,2019
1,2019-02-01,114.805,0.516888,Índice general,peninsular,2019
2,2019-03-01,126.515,0.541843,Índice general,peninsular,2019
3,2019-04-01,119.432,0.532477,Índice general,peninsular,2019
4,2019-05-01,130.280,0.552359,Índice general,peninsular,2019


In [135]:
df_estructura_generacion.head()

,fecha,valor,porcentaje,indicador,region,tipo,año
0,2019-01-01,53.722120,0.097735,Hidráulica,peninsular,Renovable,2019
1,2019-01-02,61.294768,0.086331,Hidráulica,peninsular,Renovable,2019
2,2019-01-03,82.981951,0.115989,Hidráulica,peninsular,Renovable,2019
3,2019-01-04,94.301940,0.125121,Hidráulica,peninsular,Renovable,2019
4,2019-01-05,58.822692,0.086633,Hidráulica,peninsular,Renovable,2019


In [136]:
df_fronteras.head()

,valor,porcentaje,fecha,pais,año
0,-0.208239,0.003465,2019-01-01,francia-frontera,2019
1,-2.325553,0.036857,2019-01-02,francia-frontera,2019
2,-11.123719,0.243767,2019-01-03,francia-frontera,2019
3,-17.758093,0.397978,2019-01-04,francia-frontera,2019
4,-4.267317,0.102386,2019-01-05,francia-frontera,2019


In [137]:
import os
from dotenv import load_dotenv

load_dotenv()

db_host = os.getenv("DATABASE_HOST")
db_user = os.getenv("DATABASE_USER")
db_psw = os.getenv("DATABASE_PASSWORD")
db_name = os.getenv("DATABASE_NAME")

# Conexión MySQL
conn = mysql.connector.connect(
    host=db_host,
    user=db_user,
    password=db_psw,
    database=db_name
)

cursor = conn.cursor()

In [138]:
nuevo_balance = ['fecha', 'tipo', 'energia', 'region', 'valor']
df_balance = df_balance[nuevo_balance]

df_balance.head()

,fecha,tipo,energia,region,valor
0,2019-01-01,Hidráulica,Renovable,peninsular,53.722120
1,2019-01-02,Hidráulica,Renovable,peninsular,61.294768
2,2019-01-03,Hidráulica,Renovable,peninsular,82.981951
3,2019-01-04,Hidráulica,Renovable,peninsular,94.301940
4,2019-01-05,Hidráulica,Renovable,peninsular,58.822692


In [139]:
# Subir datos balance
for _, row in df_balance.iterrows():
    cursor.execute("""
        INSERT IGNORE INTO balance (fecha, tipo, energia, region, valor)
        VALUES (%s, %s, %s, %s, %s)
    """, (row['fecha'], row['tipo'], row['energia'], row['region'], row['valor']))

conn.commit()

In [140]:
nuevo_ire = ['fecha', 'indicador', 'region', 'valor', 'porcentaje', 'año']
df_demanda_ire = df_demanda_ire[nuevo_ire]

df_demanda_ire = df_demanda_ire.drop('año', axis=1)

df_demanda_ire.head()

,fecha,indicador,region,valor,porcentaje
0,2019-01-01,Índice general,peninsular,123.969,0.541252
1,2019-02-01,Índice general,peninsular,114.805,0.516888
2,2019-03-01,Índice general,peninsular,126.515,0.541843
3,2019-04-01,Índice general,peninsular,119.432,0.532477
4,2019-05-01,Índice general,peninsular,130.280,0.552359


In [141]:
# Subir datos demanda ire

for _, row in df_demanda_ire.iterrows():
    cursor.execute("""
        INSERT IGNORE INTO demanda_ire_general (fecha, indicador, region, valor, porcentaje)
        VALUES (%s, %s, %s, %s, %s)
    """, (row['fecha'], row['indicador'], row['region'], row['valor'], row['porcentaje']))

conn.commit()

In [145]:
nuevo_demanda = ['fecha', 'indicador', 'region', 'valor']
df_demanda_evolucion = df_demanda_evolucion[nuevo_demanda]
df_demanda_evolucion.head()

,fecha,indicador,region,valor
0,2019-01-01,Demanda,peninsular,582.949806
1,2019-01-02,Demanda,peninsular,742.199407
2,2019-01-03,Demanda,peninsular,787.764963
3,2019-01-04,Demanda,peninsular,796.010035
4,2019-01-05,Demanda,peninsular,721.029651


In [143]:
# subir datos demanda evolucion

for _, row in df_demanda_evolucion.iterrows():
    cursor.execute("""
        INSERT IGNORE INTO demanda_evolucion (fecha, indicador, region, valor)
        VALUES (%s, %s, %s, %s)
    """, (row['fecha'], row['indicador'], row['region'], row['valor']))

conn.commit()

In [146]:
nuevo_fronteras = ['fecha', 'pais', 'valor', 'porcentaje', 'año']
df_fronteras = df_fronteras[nuevo_fronteras]
df_fronteras = df_fronteras.drop('año', axis=1)
df_fronteras.head()

,fecha,pais,valor,porcentaje
0,2019-01-01,francia-frontera,-0.208239,0.003465
1,2019-01-02,francia-frontera,-2.325553,0.036857
2,2019-01-03,francia-frontera,-11.123719,0.243767
3,2019-01-04,francia-frontera,-17.758093,0.397978
4,2019-01-05,francia-frontera,-4.267317,0.102386


In [147]:
# Subir datos itercambio
for _, row in df_fronteras.iterrows():
    cursor.execute("""
        INSERT IGNORE INTO fronteras (fecha, pais, valor, porcentaje)
        VALUES (%s, %s, %s, %s)
    """, (row['fecha'], row['pais'], row['valor'], row['porcentaje']))

conn.commit()

In [148]:
nuevo_estructura = ['fecha', 'indicador', 'region', 'tipo', 'valor', 'porcentaje', 'año']
df_estructura_generacion = df_estructura_generacion[nuevo_estructura]
df_estructura_generacion = df_estructura_generacion.drop('año', axis=1)
df_estructura_generacion.head()

,fecha,indicador,region,tipo,valor,porcentaje
0,2019-01-01,Hidráulica,peninsular,Renovable,53.722120,0.097735
1,2019-01-02,Hidráulica,peninsular,Renovable,61.294768,0.086331
2,2019-01-03,Hidráulica,peninsular,Renovable,82.981951,0.115989
3,2019-01-04,Hidráulica,peninsular,Renovable,94.301940,0.125121
4,2019-01-05,Hidráulica,peninsular,Renovable,58.822692,0.086633


In [149]:
# Subir datos generacion
for _, row in df_estructura_generacion.iterrows():
    cursor.execute("""
        INSERT IGNORE INTO estructura_generacion (fecha, indicador, region, tipo, valor, porcentaje)
        VALUES (%s, %s, %s, %s, %s, %s)
    """, (row['fecha'], row['indicador'], row['region'], row['tipo'], row['valor'], row['porcentaje']))

conn.commit()

Descargar Tablas de la base de datos

In [150]:
### BALANCE ###
query = "SELECT * FROM balance ORDER BY fecha DESC"
cursor.execute(query)
datos_balance = cursor.fetchall()
cursor.execute(query)
resultados = cursor.fetchall()

df = pd.DataFrame(resultados)
display(df)

,0,1,2,3,4,5
0,41223,2024-12-31,Hidráulica,Renovable,peninsular,89.12
1,41222,2024-12-30,Hidráulica,Renovable,peninsular,94.44
2,41221,2024-12-29,Hidráulica,Renovable,peninsular,81.77
3,41220,2024-12-28,Hidráulica,Renovable,peninsular,89.53
4,41219,2024-12-27,Hidráulica,Renovable,peninsular,86.65
...,...,...,...,...,...,...
2187,5,2019-01-05,Hidráulica,Renovable,peninsular,58.82
2188,4,2019-01-04,Hidráulica,Renovable,peninsular,94.30
2189,3,2019-01-03,Hidráulica,Renovable,peninsular,82.98
2190,2,2019-01-02,Hidráulica,Renovable,peninsular,61.29


In [151]:
### DEMANDA ###
query = "SELECT * FROM demanda_evolucion ORDER BY fecha DESC"
cursor.execute(query)
datos_demanda_evolucion = cursor.fetchall()

df = pd.DataFrame(datos_demanda_evolucion)
display(df)

,0,1,2,3,4
0,1461,2022-12-31,Demanda,peninsular,559.75
1,1460,2022-12-30,Demanda,peninsular,624.06
2,1459,2022-12-29,Demanda,peninsular,637.18
3,1458,2022-12-28,Demanda,peninsular,629.99
4,1457,2022-12-27,Demanda,peninsular,612.14
...,...,...,...,...,...
1456,5,2019-01-05,Demanda,peninsular,721.03
1457,4,2019-01-04,Demanda,peninsular,796.01
1458,3,2019-01-03,Demanda,peninsular,787.76
1459,2,2019-01-02,Demanda,peninsular,742.20


In [152]:
### IRE ###
query = "SELECT * FROM demanda_ire_general ORDER BY fecha DESC"
cursor.execute(query)
datos_ire_general = cursor.fetchall()

df = pd.DataFrame(datos_ire_general)
display(df)

,0,1,2,3,4,5
0,108,2023-12-01,Índice general,peninsular,97.33,0.53
1,120,2023-12-01,Índice general corregido,peninsular,85.42,0.47
2,228,2023-12-01,Índice industria,peninsular,87.72,0.54
3,240,2023-12-01,Índice industria corregido,peninsular,75.26,0.46
4,348,2023-12-01,Índice servicios,peninsular,108.87,0.52
...,...,...,...,...,...,...
355,13,2019-01-01,Índice general corregido,peninsular,105.07,0.46
356,121,2019-01-01,Índice industria,peninsular,123.06,0.55
357,133,2019-01-01,Índice industria corregido,peninsular,102.12,0.45
358,241,2019-01-01,Índice servicios,peninsular,123.36,0.53


In [153]:
### GENERACION ###
query = "SELECT * FROM estructura_generacion ORDER BY fecha DESC"
cursor.execute(query)
datos_estructura = cursor.fetchall()

df = pd.DataFrame(datos_estructura)
display(df)

,0,1,2,3,4,5,6
0,31077,2024-12-31,Solar fotovoltaica,peninsular,Renovable,77.48,0.12
1,31443,2024-12-31,Solar térmica,peninsular,Renovable,4.00,0.01
2,31809,2024-12-31,Otras renovables,peninsular,Renovable,9.61,0.01
3,32175,2024-12-31,Cogeneración,peninsular,No-Renovable,44.22,0.07
4,32541,2024-12-31,Residuos no renovables,peninsular,No-Renovable,3.55,0.01
...,...,...,...,...,...,...,...
32902,3658,2019-01-01,Solar térmica,peninsular,Renovable,7.46,0.01
32903,4021,2019-01-01,Otras renovables,peninsular,Renovable,8.43,0.02
32904,4386,2019-01-01,Cogeneración,peninsular,No-Renovable,65.18,0.12
32905,4751,2019-01-01,Residuos no renovables,peninsular,No-Renovable,5.88,0.01


In [154]:
### INTERCAMBIO ###
query = "SELECT * FROM fronteras ORDER BY fecha DESC"
cursor.execute(query)
datos_fronteras = cursor.fetchall()

df = pd.DataFrame(datos_fronteras)
display(df)

,0,1,2,3,4
0,6576,2024-12-31,marruecos-frontera,-1.02,0.21
1,8768,2024-12-31,andorra-frontera,-1.36,1.00
2,2192,2024-12-31,francia-frontera,-0.96,0.01
3,4384,2024-12-31,portugal-frontera,-57.08,0.98
4,2191,2024-12-30,francia-frontera,-0.69,0.01
...,...,...,...,...,...
8763,4386,2019-01-02,marruecos-frontera,-0.64,0.14
8764,1,2019-01-01,francia-frontera,-0.21,0.00
8765,2193,2019-01-01,portugal-frontera,-28.72,0.86
8766,4385,2019-01-01,marruecos-frontera,-0.42,0.06


In [155]:
cursor.close()
conn.close()